# Post-infection immunity
Considerations around the extent of immunity are critical in determining 
the long-term dynamics of infectious diseases.

## Structural assumptions regarding immunity
Let's have a brief look at the following structural assumptions
we could make around the extent and duration of post-infection immunity.
These can be termed SI, SIR, SIS and SIRS.
![](../images/immunity_structures.svg)

In [ ]:
# If running on Google Colab, run the following line of code to install the summer package
# %pip install summerepi2

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def get_si_base_structure(
    model_config: dict,
    extra_comps=[],
) -> CompartmentalModel:
    """
    Generate a mode that doesn't do much in itself, but has some basic
    characteristics that we can then use to add in different assumptions
    around post-infection immunity.
    We don't apply the infection process yet, as the destination compartment
    for infection will be determined later.
    
    Args:
        model_config: The fixed values used in creating the model structure
        extra_comps: Any compartments to incorporate in addition to the base ones
    Returns:
        The summer model object
    """
    
    # Compartments are comprised of the base ones and any additional latency compartments requested
    compartments = [
        "susceptible",
        "infectious",
    ] + extra_comps
    
    # Otherwise the model is very similar to that from notebook 01, as follows
    infectious_compartment = ["infectious"]
    analysis_times = (
        model_config["start_time"], 
        model_config["end_time"],
    )
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=infectious_compartment,
    )
    pop = model_config["population"]
    seed = model_config["seed"]
    suscept_pop = pop - seed
    msg = "Seed larger than population"
    assert pop >= 0., msg
    model.set_initial_population(
        distribution={
            "susceptible": suscept_pop, 
            "infectious": seed}
    )
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("contact_rate"),
        source="susceptible", 
        dest="infectious",
    )
    return model

In [ ]:
config = {
    "population": 1.,
    "seed": 0.001,
    "start_time": 0.,
    "end_time": 40.,
}

parameters = {
    "contact_rate": 1.,
    "recovery_rate": 0.333,
    "death_rate": 0.05,
}

### SI structure
We can represent permanent infection and infectiousness
by ensuring that anyone entering the `infectious` compartment
remains forever trapped within this state.
There are few infectious diseases that would be accurately
represented with this structural assumption.

In [ ]:
si_model = get_si_base_structure(config, [])

si_model.run(parameters=parameters)
si_values = si_model.get_outputs_df()
si_values.plot()

### SIR structure
For context, let's next consider the model we have been
using throughout several of the preceding notebooks.
Here we assume that immunity to reinfection is 
permanent and complete, which may be appropriate
for some infectious diseases.

In [ ]:
sir_model = get_si_base_structure(config, ["recovered"])

sir_model.add_transition_flow(
    name="recovery", 
    fractional_rate=Parameter("recovery_rate"), 
    source="infectious", 
    dest="recovered",
)

sir_model.run(parameters=parameters)
sir_values = sir_model.get_outputs_df()
sir_values.plot()

### SIS structure
Under this structure,
no immunity is conferred by infection.
That is, recovered individuals are at the same
risk of reinfection as those who have never been infected.

In [ ]:
sis_model = get_si_base_structure(config)

sis_model.add_transition_flow(
    name="recovery", 
    fractional_rate=Parameter("recovery_rate"), 
    source="infectious", 
    dest="susceptible",
)

sis_model.run(parameters=parameters)
sis_values = sis_model.get_outputs_df()
sis_values.plot()

### SIRS structure
Under this assumption,
complete immunity is obtained for a limited period
after recovery from the infectious state.
After an initial epidemic wave depletes the susceptible population,
the model approaches an equilibrium state in which
the rate of infection offsets the rate of waning of immunity
from the recovered population.

In [ ]:
parameters.update(
    {"immunity_waning": 0.1}
)
sirs_model = get_si_base_structure(config, ["recovered"])

sirs_model.add_transition_flow(
    name="recovery", 
    fractional_rate=Parameter("recovery_rate"), 
    source="infectious", 
    dest="recovered",
)

sirs_model.add_transition_flow(
    name="immunity_waning",
    fractional_rate=Parameter("immunity_waning"),
    source="recovered",
    dest="susceptible",
)

sirs_model.run(parameters=parameters)
sirs_values = sirs_model.get_outputs_df()
sirs_values.plot()

## Tracking immunity status
One really important consideration whenever constructing
compartmental models of infectious disease transmission
is that these models are "memory-less".
That is, the model state at future time points are entirely
determined by the model's current state.
Therefore, although we can calculate the rate of 
new persons transitioning between two given model states
at a certain point in time,
calculating these rates does not provide us with information
about the history of the new arrivals into the destination compartment.
If we wish to obtain this sort of information from a compartmental model,
this can be achieved by incorporating additional compartments to track
past states.
For example, consider an alternative structure to the SIRS
assumption around waning immunity.
![](../images/sirs2_structure.svg)

In [ ]:
parameters.update(
    {"immunity_waning": 0.1}
)
sirs2_model = get_si_base_structure(config, ["recovered", "susceptible_2"])

sirs2_model.add_infection_frequency_flow(
    name="reinfection",
    contact_rate=Parameter("contact_rate"),
    source="susceptible_2", 
    dest="infectious",
)
    
sirs2_model.add_transition_flow(
    name="recovery", 
    fractional_rate=Parameter("recovery_rate"), 
    source="infectious", 
    dest="recovered",
)

sirs2_model.add_transition_flow(
    name="immunity_waning",
    fractional_rate=Parameter("immunity_waning"),
    source="recovered",
    dest="susceptible_2",
)

sirs2_model.run(parameters=parameters)
sirs2_values = sirs2_model.get_outputs_df()
sirs2_values.plot()

Note that the dynamics of this model are identical
to those of the SIRS model in which recovery
returned infectious persons to the starting susceptible compartment.
The only difference is that we now have two compartments
to represent the susceptible population together,
and so could produce outputs for quantities including 
the proportion of the total population ever infected 
and the proportion of infections attributable to reinfection.
These quantities could not have been obtained from
the simpler SIRS model introduced above.

## Comparison
In summary, let's look at the dynamics of each of the
model structures we have considered.
(Toggle the SIRS2 line off to see the SIRS line underneath.)

In [ ]:
pd.DataFrame(
    {
        "si": si_values["infectious"],
        "sir": sir_values["infectious"],
        "sis": sis_values["infectious"],
        "sirs": sirs_values["infectious"],
        "sirs2": sirs2_values["infectious"],
    }
).plot(title="Prevalence under various immunity assumptions")